In [1]:
import time

import numpy as np

from mindspore import context, nn, Tensor, ops, jit, set_seed, data_sink
from mindspore import dtype as mstype
from sympy import Function, symbols, sin, cos, pi
from mindflow.utils import load_yaml_config
from mindflow.cell import FCSequential
from mindflow.pde import PDEWithLoss, sympy_to_mindspore


from src import create_training_dataset, create_test_dataset
from src import calculate_l2_error

set_seed(123456)
np.random.seed(123456)

In [2]:
# set context for training: using graph mode for high performance training with GPU acceleration
config = load_yaml_config("configs/darcy.yaml")
context.set_context(mode=context.GRAPH_MODE, device_target="GPU", device_id=0)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

In [3]:
# create train dataset
geom_name = "flow_region"
flow_train_dataset = create_training_dataset(config, geom_name)
train_data = flow_train_dataset.create_dataset(
    batch_size=config["data"]["train"]["batch_size"], shuffle=True, drop_remainder=True
)

# create test dataset
test_input, test_label = create_test_dataset(config)

In [4]:
# network model
model = FCSequential(in_channels=config["model"]["in_channels"],
                     out_channels=config["model"]["out_channels"],
                     neurons=config["model"]["neurons"],
                     layers=config["model"]["layers"],
                     residual=config["model"]["residual"],
                     act=config["model"]["activation"],
                     weight_init=config["model"]["weight_init"])

In [5]:
# optimizer
params = model.trainable_params()
optimizer = nn.Adam(params, learning_rate=config["optimizer"]["learning_rate"])

In [6]:
class Darcy2D(PDEWithLoss):
    def __init__(self, model, loss_fn=nn.MSELoss()):
        self.x, self.y = symbols("x y")
        self.u = Function("u")(self.x, self.y)
        self.v = Function("v")(self.x, self.y)
        self.p = Function("p")(self.x, self.y)
        self.in_vars = [self.x, self.y]
        self.out_vars = [self.u, self.v, self.p]
        self.loss_fn = loss_fn
        self.bc_nodes = sympy_to_mindspore(self.bc(), self.in_vars, self.out_vars)
        super(Darcy2D, self).__init__(model, self.in_vars, self.out_vars)

    def force_function(self, x, y):
        return 8 * pi**2 * sin(2 * pi * x) * cos(2 * pi * y)

    def pde(self):
        loss_1 = (
            self.u.diff(self.x)
            + self.v.diff(self.y)
            - self.force_function(self.x, self.y)
        )
        loss_2 = self.u + self.p.diff(self.x)
        loss_3 = self.v + self.p.diff(self.y)
        return {"loss_1": loss_1, "loss_2": loss_2, "loss_3": loss_3}

    def bc(self):
        u_boundary = self.u - (-2 * pi * cos(2 * pi * self.x) * cos(2 * pi * self.y))

        v_boundary = self.v - (2 * pi * sin(2 * pi * self.x) * sin(2 * pi * self.y))

        p_boundary = self.p - (sin(2 * pi * self.x) * cos(2 * pi * self.y))

        return {
            "u_boundary": u_boundary,
            "v_boundary": v_boundary,
            "p_boundary": p_boundary,
        }

    def get_loss(self, pde_data, bc_data):
        pde_res = ops.Concat(1)(self.parse_node(self.pde_nodes, inputs=pde_data))
        pde_loss = self.loss_fn(
            pde_res, Tensor(np.array([0.0]).astype(np.float32), mstype.float32)
        )

        bc_res = ops.Concat(1)(self.parse_node(self.bc_nodes, inputs=bc_data))
        bc_loss = self.loss_fn(
            bc_res, Tensor(np.array([0.0]).astype(np.float32), mstype.float32)
        )

        return pde_loss + bc_loss

In [7]:
def train():
    # define problem
    problem = Darcy2D(model)

    # prepare loss scaler
    if use_ascend:
        from mindspore.amp import DynamicLossScaler, all_finite, auto_mixed_precision
        loss_scaler = DynamicLossScaler(1024, 2, 100)
        auto_mixed_precision(model, 'O3')
    else:
        loss_scaler = None

    def forward_fn(pde_data, bc_data):
        loss = problem.get_loss(pde_data, bc_data)
        if use_ascend:
            loss = loss_scaler.scale(loss)
        return loss

    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

    @jit
    def train_step(pde_data, bc_data):
        loss, grads = grad_fn(pde_data, bc_data)
        if use_ascend:
            loss = loss_scaler.unscale(loss)
            is_finite = all_finite(grads)
            if is_finite:
                grads = loss_scaler.unscale(grads)
                loss = ops.depend(loss, optimizer(grads))
            loss_scaler.adjust(is_finite)
        else:
            loss = ops.depend(loss, optimizer(grads))
        return loss

    epochs = config["data"]["train"]["epochs"]
    steps_per_epochs = train_data.get_dataset_size()
    sink_process = data_sink(train_step, train_data, sink_size=1)

    for epoch in range(1, 1 + epochs):
        local_time_beg = time.time()
        model.set_train(True)
        for _ in range(steps_per_epochs):
            cur_loss = sink_process()
        print(f"epoch: {epoch} train loss: {cur_loss} epoch time: {(time.time() - local_time_beg) * 1000 :.3f} ms")
        model.set_train(False)
        if epoch % config["summary"]["eval_interval_epochs"] == 0:
            calculate_l2_error(model, test_input, test_label, config["data"]["train"]["batch_size"])

In [8]:
start_time = time.time()
train()
print("End-to-End total time: {} s".format(time.time() - start_time))

u_boundary: u(x, y) + 2*pi*cos(2*pi*x)*cos(2*pi*y)
    Item numbers of current derivative formula nodes: 2
v_boundary: v(x, y) - 2*pi*sin(2*pi*x)*sin(2*pi*y)
    Item numbers of current derivative formula nodes: 2
p_boundary: p(x, y) - sin(2*pi*x)*cos(2*pi*y)
    Item numbers of current derivative formula nodes: 2
loss_1: -8*pi**2*sin(2*pi*x)*cos(2*pi*y) + Derivative(u(x, y), x) + Derivative(v(x, y), y)
    Item numbers of current derivative formula nodes: 3
loss_2: u(x, y) + Derivative(p(x, y), x)
    Item numbers of current derivative formula nodes: 2
loss_3: v(x, y) + Derivative(p(x, y), y)
    Item numbers of current derivative formula nodes: 2


[ERROR] CORE(212411,7fe54c554740,python):2024-06-11-22:10:32.521.663 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_212411/1475634962.py]
[ERROR] CORE(212411,7fe54c554740,python):2024-06-11-22:10:32.522.684 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_212411/1475634962.py]
[ERROR] CORE(212411,7fe54c554740,python):2024-06-11-22:10:32.524.137 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_212411/1475634962.py]
[ERROR] CORE(212411,7fe54c554740,python):2024-06-11-22:10:32.526.768 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_212411/1475634962.py]
[ERROR] CORE(212411,7fe54c554740,python):2024-06-11-22:10:32.527.716 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_212411/1475634962.py]
[ERROR] CORE(212411,7fe54c554740,python):2024-06-11-22:10:32.529.084 [

epoch: 1 train loss: 523.6344 epoch time: 6663.886 ms
epoch: 2 train loss: 521.4643 epoch time: 2568.312 ms
epoch: 3 train loss: 522.4199 epoch time: 2585.766 ms
epoch: 4 train loss: 530.79877 epoch time: 2535.153 ms
epoch: 5 train loss: 526.6251 epoch time: 2560.666 ms
epoch: 6 train loss: 531.1123 epoch time: 2515.944 ms
epoch: 7 train loss: 522.90784 epoch time: 2708.786 ms
epoch: 8 train loss: 529.28906 epoch time: 2966.518 ms
epoch: 9 train loss: 527.5852 epoch time: 2728.937 ms
epoch: 10 train loss: 523.2656 epoch time: 2703.769 ms
epoch: 11 train loss: 522.587 epoch time: 2509.272 ms
epoch: 12 train loss: 526.1701 epoch time: 2532.090 ms
epoch: 13 train loss: 531.89575 epoch time: 2522.648 ms
epoch: 14 train loss: 530.9858 epoch time: 2505.418 ms
epoch: 15 train loss: 529.33984 epoch time: 2471.594 ms
epoch: 16 train loss: 526.0478 epoch time: 2524.454 ms
epoch: 17 train loss: 522.0132 epoch time: 2769.181 ms
epoch: 18 train loss: 531.60345 epoch time: 2566.707 ms
epoch: 19 trai

In [9]:
from mindspore import save_checkpoint

save_checkpoint(model, "darcy2D.ckpt")